In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pip install pytorch-lightning -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.4/716.4 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 26.7 MB/s eta 0:00:00


In [3]:
# Preprocess the Penn Treebank dataset
# Load the dataset
import torch

def read_data(file_path):
    parsed_data = []
    with open(file_path, 'r', encoding='utf-8') as f:
      parsed_tokens = []
      for line in f:
            if line == '\n' or line[0] == '#':
                if parsed_tokens:
                  parsed_data.append(parsed_tokens)
                parsed_tokens = []
            else:
                parts = line.strip().split(' _ _ ')
                word = parts[0]
                tag = parts[1] if len(parts) > 1 else ''
                parsed_tokens.append((word, tag))
    return parsed_data

train_dataset= read_data('/content/drive/MyDrive/DeepLearning/multiconer2023/BN-Bangla/bn_train.conll')
dev_dataset= read_data('/content/drive/MyDrive/DeepLearning/multiconer2023/BN-Bangla/bn_dev.conll')
test_dataset= read_data('/content/drive/MyDrive/DeepLearning/multiconer2023/BN-Bangla/bn_test.conll')

for i in range(100):
    print(train_dataset[i])
 

[('স্টেশনটি', 'O'), ('প্ল্যাটফর্ম', 'B-OtherPROD'), ('স্ক্রিন', 'I-OtherPROD'), ('ডোর', 'I-OtherPROD'), ('দিয়ে', 'O'), ('সজ্জিত।', 'O')]
[('উদ্ভিদটির', 'O'), ('৯০', 'O'), ('০০০', 'O'), ('এরও', 'O'), ('বেশি', 'O'), ('সৌর', 'B-OtherPROD'), ('প্যানেল', 'I-OtherPROD'), ('২৩৫', 'O'), ('একরও', 'O'), ('বেশি', 'O'), ('রয়েছে।', 'O')]
[('এই', 'O'), ('মিশনটি', 'O'), ('চন্দ্র', 'B-OtherPROD'), ('এক্স-রশ্মি', 'I-OtherPROD'), ('মানমন্দির', 'I-OtherPROD'), ('মোতায়েন', 'O'), ('করেছে।', 'O')]
[('সংস্থাটি', 'O'), ('বেশ', 'O'), ('কয়েকটি', 'O'), ('বিমান', 'O'), ('এবং', 'O'), ('রাডার', 'B-OtherPROD'), ('স্কোয়াড্রনগুলিতে', 'O'), ('কমান্ড', 'O'), ('এবং', 'O'), ('নিয়ন্ত্রণ', 'O'), ('সরবরাহ', 'O'), ('করেছে।', 'O')]
[('ট্রাম', 'B-OtherPROD'), ('স্টেশনটি', 'O'), ('কোচি', 'B-HumanSettlement'), ('কোওচি', 'B-ORG'), ('প্রশাসনিক', 'I-ORG'), ('অঞ্চল', 'I-ORG'), ('সূর্য', 'B-HumanSettlement'), ('উদয়ের', 'I-HumanSettlement'), ('দেশ', 'I-HumanSettlement'), ('।', 'O')]
[('একটি', 'O'), ('কোয়ান্টাম', 'B-OtherPROD'),

In [4]:
SEQ_LEN = 30

# Create word_to_idx and tag_to_idx mappings
word_to_idx = {"<PAD>": 0, "<UNK>": 1}
tag_to_idx = {"<PAD>": 0}


def preprocess(dataset):
    # Extract sentences and tags
    sent = [[token.lower() for token, tag in sentence] for sentence in dataset]
    tags = [[tag for token, tag in sentence] for sentence in dataset]

    for i in range(len(sent)):
        while len(sent[i]) < SEQ_LEN:
            sent[i].append('<PAD>')
            tags[i].append('<PAD>')

        if len(sent[i]) > SEQ_LEN:
            sent[i] = sent[i][:SEQ_LEN]
            tags[i] = tags[i][:SEQ_LEN]
    
    for sentence_tags in tags:
        for tag in sentence_tags:
            if tag not in tag_to_idx:
                tag_to_idx[tag] = len(tag_to_idx)
    
    for sentence in sent:
        for word in sentence:
            if word not in word_to_idx:
                word_to_idx[word] = len(word_to_idx)

    # Convert words and tags to indices
    X = torch.tensor([[word_to_idx.get(word, 1) for word in sentence] for sentence in sent], dtype=torch.int).type(torch.LongTensor)
    Y = torch.tensor([[tag_to_idx[tag] for tag in sentence] for sentence in tags], dtype=torch.int).type(torch.LongTensor)
    
    return X, Y

In [5]:
train_X, train_Y = preprocess(train_dataset)
dev_X, dev_Y = preprocess(dev_dataset)
test_X, test_Y = preprocess(test_dataset)

In [6]:
# Print the sizes of the datasets
print(f"Number of training examples: {len(train_X)}")
print(f"Number of validation examples: {len(dev_X)}")
print(f"Number of testing examples: {len(test_X)}")
# for i in range(100):
#     print(train_X[i])

Number of training examples: 9708
Number of validation examples: 507
Number of testing examples: 19859


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl

class NERModel(pl.LightningModule):
    def __init__(self, vocab_size, tagset_size, embedding_dim, hidden_dim, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim) #B * seq_len, B * seq_len * embedding_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers=num_layers, bidirectional=True)
        self.fc = nn.Linear(2*hidden_dim, tagset_size)
        self.loss_fn = nn.CrossEntropyLoss()
    
    def forward(self, x):
        embeds = self.embedding(x)
        #print(embeds.shape)
        lstm_out, _ = self.lstm(embeds)
        tag_space = self.fc(lstm_out)
        tag_scores = nn.functional.log_softmax(tag_space, dim=2)
        return tag_scores
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('test_loss', loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters())
        return optimizer

In [8]:
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

EMBEDDING_DIM = 125
HIDDEN_DIM    = 125
NUM_EPOCHS    = 10 
BATCH_SIZE    = 10

train_dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = TensorDataset(dev_X, dev_Y)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_X, test_Y)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [9]:
model = NERModel(vocab_size=len(word_to_idx), tagset_size=len(tag_to_idx), embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM)
early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")
trainer = pl.Trainer(max_epochs=NUM_EPOCHS, callbacks=[early_stopping])
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

trainer.test(dataloaders=test_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 5.3 M 
1 | lstm      | LSTM             | 252 K 
2 | fc        | Linear           | 17.1 K
3 | loss_fn   | CrossEntropyLoss | 0     
-----------------------------------------------
5.6 M     Trainable params
0         Non-trainable params
5.6 M     Total params
22.385    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_0/checkpoints/epoch=6-step=6797.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_0/checkpoints/epoch=6-step=6797.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.2563086748123169     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.2563086748123169}]

In [10]:
from sklearn.metrics import classification_report

# define idx_to_tag
idx_to_tag = {idx: tag for tag, idx in tag_to_idx.items()}

# define device
device = torch.device('cpu')

# Create a dataloader for the test set
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Set the model to evaluation mode
model.eval()

y_true = []
y_pred = []

with torch.no_grad():
    for x, y in test_loader:
        # Move the data to the device
        x = x.to(device)
        y = y.to(device)

        # Forward pass
        y_hat = model(x)

        # Compute the predicted tags
        y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

        # Compute the true tags
        y_true += [idx_to_tag[i] for i in y.cpu().numpy().flatten().tolist()]

print(classification_report(y_true, y_pred))

                         precision    recall  f1-score   support

                  <PAD>       1.00      1.00      1.00    339381
B-AerospaceManufacturer       0.16      0.05      0.08        97
  B-AnatomicalStructure       0.67      0.58      0.62       532
              B-ArtWork       0.21      0.01      0.02       455
               B-Artist       0.47      0.39      0.43      2744
              B-Athlete       0.37      0.38      0.38      1087
      B-CarManufacturer       0.64      0.90      0.75        84
               B-Cleric       0.50      0.59      0.54       240
             B-Clothing       0.22      0.65      0.33        17
              B-Disease       0.71      0.66      0.68       554
                B-Drink       0.68      0.80      0.74       120
             B-Facility       0.56      0.47      0.51       894
                 B-Food       0.45      0.39      0.42       453
      B-HumanSettlement       0.77      0.67      0.71      6011
     B-MedicalProcedure 

In [11]:
# Set the model to evaluation mode
model.eval()

idx_to_word = {idx: word for word, idx in word_to_idx.items()}

y_true = []
y_pred = []

with torch.no_grad():
    for x, y in test_loader:
        # Move the data to the device
        x = x.to(device)
        y = y.to(device)

        # Forward pass
        y_hat = model(x)

        # Get back the sentence
        x_sent = [idx_to_word[i] for i in x.cpu().numpy().flatten().tolist()]

        # Compute the predicted tags
        y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

        # Compute the true tags
        y_true += [idx_to_tag[i] for i in y.cpu().numpy().flatten().tolist()]
        print("Sentence\t:", end = ' ')
        for i in range(len(x_sent)):
          print(x_sent[i], end = ' ')
        print("\nPredicted tags\t:", end = ' ')
        for i in range(len(y_pred)):
          print(y_pred[i], end = ' ')
        break

Sentence	: প্রোপেলারটি একটি ডি হ্যাভিল্যান্ড এয়ারক্রাফ্ট কোম্পানি স্থির পিচ টাইপ ছিল। <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> এটি ১৯৫৫ সালে ১৯৫৭ সালে নর্ড এভিয়েশন পরে snias দ্বারা ডিজাইন করা হয়েছিল। <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> পাঁচটি f108 চালিত উদাহরণগুলি বোয়িং কোম্পানি থেকে সরাসরি অনুরোধ করা হয়েছিল বাকী অন্যান্য পক্ষ থেকে। <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> গ্রিনহিল ১৯৯৫ সালে l'auto-neige bombardier limitée যোগদানের আগে mckinsey & company, inc. এ তাঁর কেরিয়ার শুরু করেছিলেন। <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> যদিও ডকটি অ্যাসোসিয়েটেড ব্রিটিশ পোর্টস এর নিয়ন্ত্রণে পড়ে এটি বর্তমানে কেবলমাত্র bae সিস্টেম পিএলসি দ্বারা ব্যবহৃত হয়। <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> তিনি বিদেশী অন্যান্য চুক্তি

In [12]:
torch.save(model.state_dict(), '/content/drive/MyDrive/DeepLearning/multiconer2023/BN-Bangla/Bangla_fine.pth')

In [13]:
bangla_model = model.load_state_dict(torch.load('/content/drive/MyDrive/DeepLearning/multiconer2023/BN-Bangla/Bangla_fine.pth'))

def pred_NER(sentence):
    # Split the sentence into words
    words = sentence.split()
    for word in words:
      word = word.lower()

    # Get the index of the words
    X = []
    for word in words:
        if word not in word_to_idx:
            X.append(word_to_idx["<UNK>"])
        else:
            X.append(word_to_idx[word])
    
    # Create a tensor
    X = torch.tensor(X).unsqueeze(0)
    
    # Get the predictions
    y_hat = model(X)
    
    # Get the predictions
    predictions = [idx_to_tag[pred] for pred in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

    # Give new sentence
    new_sentence = []
    flag = False
    tag = ""
    for word, pred in zip(words, predictions):
        if pred == "O":
            if flag:
                new_sentence[new_sentence.__len__() - 1] = new_sentence[new_sentence.__len__() - 1] + "[" + tag + "]"
                flag = False
                tag = ""
            new_sentence.append(word)
        elif len(pred) == 0:
            new_sentence.append(word)
        elif pred[0] == "B":
            flag = True
            tag = pred[2:]
            new_sentence.append(word)
        else:
            new_sentence.append(word)
    if flag:
        new_sentence[new_sentence.__len__() - 1] = new_sentence[new_sentence.__len__() - 1] + "[" + tag + "]"
        flag = False
        tag = "" 
    
    return " ".join(new_sentence)
print(pred_NER("আমি জিম ভালোবাসি"))


আমি জিম ভালোবাসি[Athlete]
